In [1]:
import pandas as pd
import numpy as np

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms

In [102]:
## Load Data

train_input = pd.read_csv('../data/data_train.csv')
train_labels = train_input['labels'].values
# train_labels = pd.get_dummies(train_input['labels']).as_matrix()
del train_input['labels']
train_input = train_input.as_matrix()
# print(train_input.shape)
# print(train_labels.shape)

test_input = pd.read_csv("../data/data_val.csv")
test_labels = test_input['labels'].values
# test_labels = pd.get_dummies(test_input['labels']).as_matrix()
del test_input['labels']
test_input = test_input.as_matrix()
# print(test_input.shape)
# print(test_labels.shape)

In [6]:
img_height = 28
img_width = 28
img_channels = 1

n_classes = 10
batch_size = 50
epochs = 2

device_name='eren_'
model_name = device_name+'pytorch_conv_1'
vector_dimension = 10

In [104]:
train_input_, train_labels_ = torch.FloatTensor(train_input), torch.FloatTensor(train_labels)

model = torch.nn.Sequential(
          torch.nn.Linear(784, 20),
          torch.nn.ReLU(),
          torch.nn.Linear(20, 1),
        )
param = list(model.parameters())

loss_fn = torch.nn.MSELoss(size_average=False)
optimizer = optim.Adadelta(param)
batch_size = 50
learning_rate = 1e-4
for t in range(5):
    x, y = train_input_[t*batch_size:(t+1)*batch_size], train_labels_[t*batch_size:(t+1)*batch_size]

    if type(x) != torch.autograd.variable.Variable:
        x, y = Variable(x), Variable(y)
    y_pred = model(x)
    
    loss = loss_fn(y_pred, y)
    print(t, loss.data[0])
    
    model.zero_grad()
    
    loss.backward()
    optimizer.step()


0 32372.828125
1 137864.203125
2 5948.13330078125
3 4740.07421875
4 2728.708251953125


In [122]:
## Model Definition for Classification
train_input_, train_labels_ = torch.FloatTensor(train_input), torch.LongTensor(train_labels)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 10, 1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.linear = nn.Linear(13*13*10,10)
    def forward(self, x):
        x = x.contiguous()
        x = x.view(-1, img_channels, img_height, img_width)
        
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)
        x = x.view(-1, 10*13*13)
        x = self.linear(x)
        return x

net = Net()
# print(str(net))

param = list(net.parameters())
optimizer = optim.Adadelta(param)
loss_fn = torch.nn.CrossEntropyLoss()
batch_size = 50

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    # number of batches
    imax = int(train_input.shape[0]/batch_size)
    for i in range(10):
        # get the inputs
        x, y = train_input_[i*batch_size:(i+1)*batch_size], train_labels_[i*batch_size:(i+1)*batch_size]

        if type(x) != torch.autograd.variable.Variable:
            x, y = Variable(x), Variable(y)
        y_pred = net(x)

        loss = loss_fn(y_pred, y)
        print(t, loss.data[0])

        net.zero_grad()
#         optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

4 13.709769248962402
4 13.225839614868164
4 8.32754898071289
4 4.868668556213379
4 3.2344887256622314
4 2.2833399772644043
4 1.1323071718215942
4 1.2553083896636963
4 1.2968429327011108
4 0.8567118048667908
4 0.9646856784820557
4 0.26497364044189453
4 0.38566550612449646
4 0.6813852787017822
4 0.3424835503101349
4 0.41514697670936584
4 0.29375848174095154
4 0.1549941599369049
4 0.524043083190918
4 0.3139924705028534
Finished Training


In [132]:
## Model Definition for Classification

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.conv4 = nn.Conv2d(64, 64, 3)
        self.conv5 = nn.Conv2d(64, 128, 2)
        self.conv6 = nn.Conv2d(128, 10, 1)
        self.pool1 = nn.MaxPool2d(2, 2)

    def forward(self, x):
        
        x = x.contiguous()
        x = x.view(-1, img_channels, img_height, img_width)
#         print(x.shape)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)
#         print(x.shape)

        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool1(x)
#         print(x.shape)

        x = F.relu(self.conv5(x))
        x = self.pool1(x)
#         print(x.shape)

        x = F.relu(self.conv6(x))
        x = x.view(-1,10)
        return x

net = Net()
# print(str(net))

param = list(net.parameters())
optimizer = optim.Adadelta(param)
loss_fn = torch.nn.CrossEntropyLoss()
batch_size = 50


for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    # number of batches
    imax = int(train_input.shape[0]/batch_size)
    for i in range(10):
        # get the inputs
        x, y = train_input_[i*batch_size:(i+1)*batch_size], train_labels_[i*batch_size:(i+1)*batch_size]

        if type(x) != torch.autograd.variable.Variable:
            x, y = Variable(x), Variable(y)
        y_pred = net(x)

        loss = loss_fn(y_pred, y)
        print(t, loss.data[0])

        net.zero_grad()
#         optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

4 3.1925160884857178
4 4.503812789916992
4 2.301778554916382
4 2.3028433322906494
4 2.302583694458008
4 2.302583694458008
4 2.302583694458008
4 2.302583694458008
4 2.302583694458008
4 2.302583694458008
Finished Training
